In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Use the citipy module 
from citipy import citipy

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

import time
from datetime import datetime   

In [2]:
#Create a new set of 2,000 random latitudes and longitudes

# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)


In [3]:

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


774

In [4]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

    # Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
      
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | pisco
Processing Record 2 of Set 1 | almaznyy
Processing Record 3 of Set 1 | mataura
Processing Record 4 of Set 1 | hithadhoo
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | nortelandia
Processing Record 7 of Set 1 | saint george
Processing Record 8 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 9 of Set 1 | aklavik
Processing Record 10 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 11 of Set 1 | kodiak
Processing Record 12 of Set 1 | busselton
Processing Record 13 of Set 1 | qujing
Processing Record 14 of Set 1 | cape town
Processing Record 15 of Set 1 | sao filipe
Processing Record 16 of Set 1 | vaini
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | vao
Processing Record 19 of Set 1 | yellowknife
Processing Record 20 of Set 1 | longyearbyen
Processing Record 21 of Set 1 | vaitupu
City not found. Skipping

In [5]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Pisco,-13.7000,-76.2167,67.01,88,0,16.11,PE,clear sky
1,Almaznyy,48.0448,40.0450,39.18,84,14,8.19,RU,few clouds
2,Mataura,-46.1927,168.8643,56.25,44,40,5.50,NZ,scattered clouds
3,Hithadhoo,-0.6000,73.0833,82.22,66,0,1.39,MV,clear sky
4,Ushuaia,-54.8000,-68.3000,38.86,70,40,14.97,AR,scattered clouds
...,...,...,...,...,...,...,...,...,...
702,Lima,-12.0432,-77.0282,72.90,74,40,11.50,PE,scattered clouds
703,Piranshahr,36.7010,45.1413,52.30,50,98,3.87,IR,overcast clouds
704,Yashalta,46.3395,42.2762,41.95,82,3,4.12,RU,clear sky
705,Shar,49.5886,81.0453,44.58,93,99,7.96,KZ,overcast clouds


In [7]:
# Create the output file (CSV).
output_data_file = "../Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")